# **Парсинг Habr**

In [231]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from time import sleep
from tqdm import tqdm
import random

In [232]:
def parse(id):
    article = requests.get('https://habr.com/ru/article/' + str(id))
    if article.status_code == 200:
        soup = BeautifulSoup(article.text, 'html.parser')
        return soup
    else:
        None

In [233]:
article1 = parse(346198)

In [234]:
# ссылка на статью
link = article1.find_all('meta',{'property':'og:url'})[0].get('content')
link

'https://habr.com/ru/articles/346198/'

In [236]:
# заголовок статьи
title = article1.find_all('meta', {'property' : 'og:title'})[0].get('content')
title

'Как я парсил Хабр, часть 1: тренды'

In [235]:
# время чтения статьи
reading_time = article1.find('span', {'class': 'tm-article-reading-time__label'}).text
reading_time = int(re.search('\d+',reading_time).group())
reading_time

6

In [ ]:
# время публикации
datetime_published = article1.findAll('span', {'class': 'tm-article-datetime-published'})[0].find().get('datetime')
datetime_published

'2018-01-08T12:54:55.000Z'

In [ ]:
# просмотры
views = article1.findAll('span', {'class': 'tm-icon-counter__value'})[0].text
views

'23K'

In [238]:
# хабы
hubs = article1.findAll('a', {'class': 'tm-hubs-list__link'})
hubs = [i.get_text().rstrip() for i in hubs]
hubs

['Python',
 'Визуализация данных',
 'Исследования и прогнозы в IT',
 'Веб-аналитика']

In [239]:
# теги
tags = article1.findAll('a', {'class': 'tm-tags-list__link'})
tags = [i.get_text().rstrip() for i in tags]
tags

['хабрахабр', 'парсинг сайтов', 'тренды']

In [ ]:
# количество пользователей, добавивших публикацию в закладки
bookmarks = article1.findAll('span', {'class': 'bookmarks-button__counter'})[0].text
bookmarks = int(re.search('\d+',bookmarks).group())
bookmarks

121

In [ ]:
# количесчтво комментариев
comments = article1.findAll('span', {'class': 'tm-article-comments-counter-link__value'})[0].text
comments = int(re.search('\d+',comments).group())
comments

45

In [ ]:
# Количество положительных, отрицательных и общее кол-во голосов за рейтинг статьи
votes = article1.findAll('span', {'class': 'tm-votes-meter__value'})[0].get('title')
result_vote = int(article1.findAll('span', {'class': 'tm-votes-meter__value'})[0].text)
if result_vote != 0:
    possitive_votes = int(votes.split(':')[1].split('и')[0].strip().split('↑')[1])
    negative_votes = int(votes.split(':')[1].split('и')[1].strip().split('↓')[1])
else:
    possitive_votes, negative_votes = 0, 0

In [ ]:
print(result_vote, possitive_votes, negative_votes)

62 62 0


In [ ]:
# Имя автора и ник
author_username = article1.find('span', {'class': 'tm-user-info__user'}).get_text('tm-user-info__username').strip()
author_username = re.findall(r'.+', author_username)[0]
author_username

'cointegrated'

In [ ]:
# рейтинг автора
lever_score = float(article1.findAll('div', {'class': 'tm-counter-container__header'})[1].text.strip())
lever_score

-2.0

In [ ]:
# карма автора
karma = article1.findAll('div', {'class': 'tm-counter-container__header'})[0].text
karma = int(re.search('\d+',karma).group())
karma

139

In [ ]:
# текст статьи
article_text = article1.find('div', id="post-content-body")
article_text
# пока просто целиком "post-content-body" без удаления лишнего

<div id="post-content-body"><div><div class="article-formatted-body article-formatted-body article-formatted-body_version-1"><div xmlns="http://www.w3.org/1999/xhtml"><p>Когда был доеден новогодний оливье, мне стало нечего делать, и я решил скачать себе на компьютер все статьи с Хабрахабра (и смежных платформ) и поисследовать. </p><br/>
<p>Получилось несколько интересных сюжетов. Первый из них — это развитие формата и тематики статей за 12 лет существования сайта. Например, достаточно показательна динамика некоторых тем. Продолжение — под катом.</p><br/>
<img data-src="https://habrastorage.org/webt/fa/ht/ez/fahtezsb64hlor2mkatmlp4q7mq.png" src="https://habrastorage.org/r/w1560/webt/fa/ht/ez/fahtezsb64hlor2mkatmlp4q7mq.png" width="540"><a name="habracut"></a><br/>
<h2 id="process-parsinga">Процесс парсинга</h2><br/>
<p>Чтобы понять, как развивался Хабр, нужно было обойти по все его статьи и выделить из них метаинформацию (например, даты). Обход дался легко, потому что ссылки на все стат

In [ ]:
# ссылки в статье
links = [i.get('href') for i in article_text.find_all('a')]
links = list(filter(lambda item: item is not None, links))
links

['http://habrahabr.ru/post/197308/',
 'https://yadi.sk/d/JV5S4nUKUrkzsA',
 'https://disk.yandex.ru/d/xv-opls6-7Yisg']

In [ ]:
# картинки
images = [i.get('src') for i in article_text.find_all('img')]
images

['https://habrastorage.org/r/w1560/webt/fa/ht/ez/fahtezsb64hlor2mkatmlp4q7mq.png',
 'https://habrastorage.org/r/w1560/webt/ga/a1/0x/gaa10x2xrj6cz9y5mjouf6qghaw.png',
 'https://habrastorage.org/r/w1560/webt/5x/xq/wy/5xxqwy0ozkxidpiunlp59wzstzm.png',
 'https://habrastorage.org/r/w1560/webt/ka/tl/nh/katlnhwb2odlxk3p6uccsqwldcw.png',
 'https://habrastorage.org/r/w1560/webt/ck/zv/vu/ckzvvuqwr1mnhs_0hvvw0qdaqqs.png',
 'https://habrastorage.org/r/w1560/webt/ce/gj/k9/cegjk9fpdcpi7gqijes6dpzvwle.png',
 'https://habrastorage.org/r/w1560/webt/fn/ay/jl/fnayjlfpsyf3vf2wsj8vyb5g2hm.png',
 'https://habrastorage.org/r/w1560/webt/k3/_t/je/k3_tjevzru_8-tez48n0zh-hiog.png',
 'https://habrastorage.org/r/w1560/webt/0q/ji/cp/0qjicpigcx6k2aeyeak0ob-wny0.png',
 'https://habrastorage.org/r/w1560/webt/v3/wo/lw/v3wolwgn1nycharjflckycowj94.png',
 'https://habrastorage.org/r/w1560/webt/vx/s5/yy/vxs5yy1ojnjf4hlufmoi4j9fsve.png']

In [241]:
def parseArticle(soup):
    # url
    url = soup.find_all('meta',{'property':'og:url'})[0].get('content')
    # заголовок статьи
    title = soup.find_all('meta', {'property' : 'og:title'})[0].get('content')
    # хабы
    hubs = soup.findAll('a', {'class': 'tm-hubs-list__link'})
    hubs = [i.get_text().rstrip() for i in hubs]
    # теги
    tags = soup.findAll('a', {'class': 'tm-tags-list__link'})
    tags = [i.get_text().rstrip() for i in tags]
    # дата публикации
    datetime_published = soup.findAll('span', {'class': 'tm-article-datetime-published'})[0].find().get('datetime')
    # кол-во просмотров
    views = soup.findAll('span', {'class': 'tm-icon-counter__value'})[0].text
    # количество пользователей, добавивших публикацию в закладки
    bookmarks = soup.findAll('span', {'class': 'bookmarks-button__counter'})[0].text
    bookmarks = int(re.search('\d+',bookmarks).group())
    # количесчтво комментариев
    comments = soup.findAll('span', {'class': 'tm-article-comments-counter-link__value'})[0].text
    comments = int(re.search('\d+',comments).group())
    # количество положительных, отрицательных и общее кол-во голосов за рейтинг статьи
    votes = soup.findAll('span', {'class': 'tm-votes-meter__value'})[0].get('title')
    result_vote = int(soup.findAll('span', {'class': 'tm-votes-meter__value'})[0].text)
    if result_vote != 0:
        possitive_votes = int(votes.split(':')[1].split('и')[0].strip().split('↑')[1])
        negative_votes = int(votes.split(':')[1].split('и')[1].strip().split('↓')[1])
    else:
        possitive_votes, negative_votes = 0, 0
    # ник автора
    author_username = soup.find('span', {'class': 'tm-user-info__user'}).get_text('tm-user-info__username').strip()
    author_username = re.findall(r'.+', author_username)[0]
    # рейтинг автора
    lever_score = float(soup.findAll('div', {'class': 'tm-counter-container__header'})[1].text.strip())
    # карма автора
    karma = soup.findAll('div', {'class': 'tm-counter-container__header'})[0].text
    karma = int(re.search('\d+',karma).group())
    # текст статьи
    article_text = soup.find('div', id="post-content-body")
    # ссылки в статье
    links = [i.get('href') for i in article_text.find_all('a')]
    links = list(filter(lambda item: item is not None, links))
    # картинки
    images = [i.get('src') for i in article_text.find_all('img')]
    return {'link': url,
            'title':title,
            'hubs': hubs,
            'tags': tags,
            'bookmarks': bookmarks,
            'comments': comments,
            'publication date': datetime_published,
            'views': views,
            'possitive_votes': possitive_votes,
            'negative_votes': negative_votes,
            'result_vote': result_vote,
            'author_username': author_username,
            'lever_score': lever_score,
            'karma': karma,
            'article_text': article_text,
            'links': links,
            'images': images}

In [ ]:
def getArticle(soup):
    url = soup.find_all('meta',{'property':'og:url'})[0].get('content')
    if 'posts' in url.split('/'):
        return None
    else:
        parsed_article = parseArticle(soup)
        return parsed_article

Выполню парсинг статей, в ссылке которых есть значение от 750000 до 750300. Если запрос успешно выполнился и по переданной ссылке находится статья, сохраню **ссылку, название статьи, хабы, теги, дату публикации,количество просмотров, положительные и отрицательные голоса, рейтинг статьи, количество пользователей, добавивших публикацию в закладки, количество комментариев, ник автора, рейтинг и карму автора, текст статьи,ссылки и картинки из статьи.**

In [242]:
parseHabr_list = list()
for i in tqdm(range(750000, 750300)):
    p = parse(i)
    if p is None:
       continue
    else:
        article = getArticle(p)
        if article is not None:
            parseHabr_list.append(article)
    sleep(random.random())


100%|██████████| 300/300 [07:09<00:00,  1.43s/it]


In [243]:
parseHabr_df = pd.DataFrame(parseHabr_list)
parseHabr_df

,link,title,hubs,tags,bookmarks,comments,publication date,views,possitive_votes,negative_votes,result_vote,author_username,lever_score,karma,article_text,links,images
0,https://habr.com/ru/companies/sbermarket/artic...,Под капотом анимаций в React Native. Часть 2/2...,"[Блог компании СберМаркет, JavaScript, Разрабо...","[react native, анимации]",31,0,2023-08-25T11:40:08.000Z,1.6K,6,0,6,Evgen175,0.0,10,"[[[<div xmlns=""http://www.w3.org/1999/xhtml""><...",[https://habr.com/ru/companies/sbermarket/arti...,[https://habrastorage.org/r/w780q1/getpro/habr...
1,https://habr.com/ru/news/750002/,Учёные начали использовать потеющего робота дл...,"[Робототехника, Научно-популярное, Здоровье]","[аризонский университет, роботы, манекен, жара...",1,1,2023-07-24T12:17:01.000Z,807,6,0,6,maybe_elf,215.0,211,"[[[<div xmlns=""http://www.w3.org/1999/xhtml""><...",[https://news.asu.edu/20230525-solutions-meet-...,[https://habrastorage.org/r/w780q1/getpro/habr...
2,https://habr.com/ru/news/750004/,Bloomberg: для разработчиков игр стало проблем...,"[Разработка под AR и VR, AR и VR, IT-компании]","[Apple, Vision Pro, VR, AR]",1,0,2023-07-24T12:03:40.000Z,644,2,0,2,avouner,20.7,84,"[[[<div xmlns=""http://www.w3.org/1999/xhtml""><...",[https://www.bloomberg.com/news/newsletters/20...,[]
3,https://habr.com/ru/companies/yandex_praktikum...,Яндекс Практикум запустил новый курс «Автомати...,"[Блог компании Яндекс Практикум, Python, Прогр...","[автоматизация тестирования, автоматизация рут...",1,0,2023-07-24T12:08:25.000Z,799,2,0,2,MarieSuperhero1,0.2,6,"[[[<div xmlns=""http://www.w3.org/1999/xhtml""><...",[https://practicum.yandex.ru/qa-automation-eng...,[]
4,https://habr.com/ru/companies/ru_mts/articles/...,"В 100 раз быстрее Wi-Fi: Li-Fi, беспроводная с...","[Блог компании МТС, Сетевые технологии, Беспро...","[сетевые технологии, беспроводные технологии, ...",33,57,2023-07-24T12:12:39.000Z,12K,23,4,19,arturovdima,-2.0,6,"[[[<div xmlns=""http://www.w3.org/1999/xhtml""><...",[https://www.businesswire.com/news/home/202307...,[https://habrastorage.org/r/w1560/getpro/habr/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,https://habr.com/ru/news/750290/,Браузер Arc вышел из бета-теста и стал доступе...,"[Браузеры, Софт]","[Arc, the browser company, браузеры, обновлени...",3,16,2023-07-25T15:24:14.000Z,4.9K,5,1,4,daniilshat,111.2,185,"[[[<div xmlns=""http://www.w3.org/1999/xhtml""><...","[https://arc.net/, https://arc.net/credits, ht...",[https://habrastorage.org/r/w780q1/getpro/habr...
132,https://habr.com/ru/companies/timeweb/articles...,Создаем I2C Master Controller на Verilog. Идея...,"[Блог компании Timeweb Cloud, FPGA, Схемотехни...","[timeweb_статьи, fpga, i2c, i2c bus, i2c maste...",56,10,2023-07-26T08:01:01.000Z,4.9K,64,0,64,megalloid,-2.0,111,"[[[<div xmlns=""http://www.w3.org/1999/xhtml"">В...","[http://kmpu.ru/other/bus_i2c/index.html, http...",[https://habrastorage.org/r/w1560/webt/bo/e2/w...
133,https://habr.com/ru/companies/banki/articles/7...,Тестовые среды для разработчиков: успехи и фей...,"[Блог компании Банки.ру, Тестирование IT-систе...",[тестирование],25,3,2023-07-25T16:34:03.000Z,2.6K,7,0,7,Adeffeus,0.0,5,"[[[<div xmlns=""http://www.w3.org/1999/xhtml""><...","[https://www.banki.ru/, https://habr.com/ru/co...",[https://habrastorage.org/r/w1560/webt/c5/ic/d...
134,https://habr.com/ru/companies/habr_career/arti...,«Я больше не могу найти работу» — истории карь...,"[Блог компании Хабр Карьера, Карьера в IT-инду...","[хабр карьера, спецпроект хабр карьеры, карьер...",107,329,2023-07-26T08:06:03.000Z,113K,96,10,86,nastyasich,19.0,21,"[[[<div xmlns=""http://www.w3.org/1999/xhtml""><...",[https://habr.com/ru/companies/habr_career/art...,[https://habrastorage.org/r/w1560/getpro/habr/...
